<a href="https://colab.research.google.com/github/Vrinda999/healthy-heart-chatbot/blob/main/Medical_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build BioMistral Medical RAG Chatbot

I'm using BioMistral LLM a and Heart Health PDF File.

## Loading Model from Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Installing Required Packages

In [3]:
!pip install langchain sentence-transformers chromadb llama-cpp-python langchain_community pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 MB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━

## Importing Libraries

In [4]:
# Indexing

from langchain_community.document_loaders import PyPDFDirectoryLoader           # Picks up PDF Files and Reads them.
from langchain.text_splitter import RecursiveCharacterTextSplitter              # Splits text data into Multiple Chunks.
from langchain_community.embeddings import SentenceTransformerEmbeddings        # Generates Embeddings.
from langchain.vectorstores import Chroma                                       # Stores Embeddings.

# For LLM Model

from langchain_community.llms import llamacpp                                   # Loading LLM Model.

from langchain.chains import RetrievalQA, LLMChain                              # Building End to End Application,m we'll use Langchain's Chain Methodology. Question Answering Chain.

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


## Importing Document

In [5]:
loader = PyPDFDirectoryLoader("/content/drive/MyDrive/Medical Chatbot")
docs = loader.load()

In [6]:
len(docs)         # Gives Number of Pages. Each Page is considered as a Document.

95

In [7]:
docs[6]         # Will Display the Contents of Page 6.

Document(metadata={'source': '/content/drive/MyDrive/Medical Chatbot/healthyheart.pdf', 'page': 6}, page_content='2\nThese facts may seem frightening, but they need not be. The good\nnews is that you have a lot of power to protect and improve yourheart health. This guidebook will help you find out your own riskof heart disease and take steps to prevent it.\n“But,” you may still be thinking, “I take pretty good care of myself.\nI’m unlikely to get heart disease.” Yet a recent national survey showsthat only 3 percent of U.S. adults practice all of the “Big Four”habits that help to prevent heart disease: eating a healthy diet, getting regular physical activity, maintaining a healthy weight, andavoiding smoking. Many young people are also vulnerable. Arecent study showed that about two-thirds of teenagers already haveat least one risk factor for heart disease.\nEvery risk factor counts. Research shows that each individual risk\nfactor greatly increases the chances of developing heart disea

## Chunking
Dividing the Entire Text into Smallert Parts to facilitate the LLM Model.

In [8]:
# We'll takle 300 Characters at a time.
# And 50 of Them would be common between chunk → for Retaining Context.

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap = 50)
chunks = text_splitter.split_documents(docs)

In [9]:
len(chunks)

747

## Embeddings Creation

In [10]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = "INSERT_YOUR_HUGGING_FACE_ACCESS_TOKEN"

In [11]:
embeddings = SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")    # Hugging Face Model Repo Name.

<ipython-input-11-2deefdb74107>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")    # Hugging Face Model Repo Name.
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate wi

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/706k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Vector Store Creation
By providing the vector store with *Chunks*, we're enabling Keyword Search, and with *embeddings*, we can do a Vector Search.
Since We Need a Hybrid Search, we're Providing both these assets.

In [12]:
vectorstore = Chroma.from_documents(chunks, embeddings)

Testing how well the Search is working.

In [13]:
query = "Who is at risk of Heart Disease?"
search_results = vectorstore.similarity_search(query)

In [14]:
search_results

[Document(metadata={'page': 8, 'source': '/content/drive/MyDrive/Medical Chatbot/healthyheart.pdf'}, page_content='While each risk factor increases your risk of heart disease, having'),
 Document(metadata={'page': 5, 'source': '/content/drive/MyDrive/Medical Chatbot/healthyheart.pdf'}, page_content='■As early as age 45, a man’s risk of heart disease begins to rise significantly. For a woman, risk starts to increase at age 55.\n■Fifty percent of men and 64 percent of women who die suddenlyof heart disease have no previous symptoms of the disease.1Heart Disease: Why Should You Care?Heart Disease:'),
 Document(metadata={'page': 6, 'source': '/content/drive/MyDrive/Medical Chatbot/healthyheart.pdf'}, page_content='factor greatly increases the chances of developing heart disease.Moreover, the worse a particular risk factor is, the more likely youare to develop heart disease. For example, if you have high bloodpressure, the higher it is, the greater your chances of developingheart disease, i

In [15]:
retriever = vectorstore.as_retriever(search_kwargs = {"k": 5})    # Top 5 Results

In [16]:
retriever.get_relevant_documents(query)

<ipython-input-16-162c3489abc9>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents(query)


[Document(metadata={'page': 8, 'source': '/content/drive/MyDrive/Medical Chatbot/healthyheart.pdf'}, page_content='While each risk factor increases your risk of heart disease, having'),
 Document(metadata={'page': 5, 'source': '/content/drive/MyDrive/Medical Chatbot/healthyheart.pdf'}, page_content='■As early as age 45, a man’s risk of heart disease begins to rise significantly. For a woman, risk starts to increase at age 55.\n■Fifty percent of men and 64 percent of women who die suddenlyof heart disease have no previous symptoms of the disease.1Heart Disease: Why Should You Care?Heart Disease:'),
 Document(metadata={'page': 6, 'source': '/content/drive/MyDrive/Medical Chatbot/healthyheart.pdf'}, page_content='factor greatly increases the chances of developing heart disease.Moreover, the worse a particular risk factor is, the more likely youare to develop heart disease. For example, if you have high bloodpressure, the higher it is, the greater your chances of developingheart disease, i

## Final Phase
Use the search Results and Pass them to the LLM for it to Summarise the Results and Present it in the form of a human Comprehensible Answer.

In [17]:
llm = llamacpp.LlamaCpp(
    model_path = "/content/drive/MyDrive/Medical Chatbot/BioMistral-7B.Q4_K_M.gguf",
    temperature = 0.2,                      # This should be lesser, since this Chatbot is MEant to be used to Generate Factual Response.
    max_tokens = 2048,
    top_p = 1                               # Top Probability = 1.
)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /content/drive/MyDrive/Medical Chatbot/BioMistral-7B.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = hub
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llam

### Using LLM, Retriever and Query to Generate Final Response.

Creating a Prompt Template to let the LLM know how to process the query and to refer to the dcontext while generating an Answer.

```</s>``` is used as a Format Separater.


In [18]:
template = """
<|context|>
You are a Medical Assitant that follows instructions and generates accurate, Truthful and Direct responses based on the Query and Context provided.
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

In [19]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

In [20]:
prompt = ChatPromptTemplate.from_template(template)

replace the *context* placeholder with the Retriever's output.
``` | ``` signifies that a step has been completed and next has begun.

In [21]:
RAG_Chain = (
    {"context": retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

Testing by Entering a Query and Getting Response.

In [22]:
# response = RAG_Chain.invoke("Who is at risk of Heart Disease?")


llama_print_timings:        load time =    6365.43 ms
llama_print_timings:      sample time =      36.77 ms /    57 runs   (    0.65 ms per token,  1550.13 tokens per second)
llama_print_timings: prompt eval time =   37065.83 ms /    68 tokens (  545.09 ms per token,     1.83 tokens per second)
llama_print_timings:        eval time =   40385.91 ms /    56 runs   (  721.18 ms per token,     1.39 tokens per second)
llama_print_timings:       total time =   77562.79 ms /   124 tokens


In [23]:
# response

'The CDC states that heart disease is the leading cause of death for people in the United States. It is also the number one killer of men over 35 years old and women over 45 years old. To determine your risk, you should consider factors such as:'

In [26]:
import sys
while True:
  user_input = input(f"Input Query: ")
  if user_input == 'exit':
    print("Exiting...")
    sys.exit()
  if user_input == "":
    continue
  response = RAG_Chain.invoke(user_input)
  print(f"Response: {response}")

Input Query: What are the Signs of an Unhealthy Heart?


Llama.generate: 48 prefix-match hit, remaining 23 prompt tokens to eval

llama_print_timings:        load time =    6365.43 ms
llama_print_timings:      sample time =      54.79 ms /    78 runs   (    0.70 ms per token,  1423.54 tokens per second)
llama_print_timings: prompt eval time =   41628.98 ms /    24 tokens ( 1734.54 ms per token,     0.58 tokens per second)
llama_print_timings:        eval time =   57830.01 ms /    77 runs   (  751.04 ms per token,     1.33 tokens per second)
llama_print_timings:       total time =   71384.17 ms /   101 tokens


Response: The signs of an unhealthy heart can include shortness of breath, fatigue, swollen feet, weight gain, weakness, dizziness, palpitations, and a bluish color on the skin. It is important to note that these symptoms may be caused by other conditions as well, so it is best to consult with a healthcare professional for an accurate diagnosis.
Input Query: 


EOFError: 